In [1]:
# !pip install git+https://github.com/ImalshaD/DimenTuna.git

  Cloning https://github.com/ImalshaD/DimenTuna.git to /tmp/pip-req-build-ci4a65hd
  Running command git clone --filter=blob:none --quiet https://github.com/ImalshaD/DimenTuna.git /tmp/pip-req-build-ci4a65hd
  Resolved https://github.com/ImalshaD/DimenTuna.git to commit ea7b5024e300e76941aa3d0e3c89e06ec22f0c2d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dimentuna: filename=dimentuna-0.1.0-py3-none-any.whl size=12211 sha256=c417f5acf62bb3d6134c96bcfde8bbf71465d2d6d5c098e9f6fb2295ddc970df
  Stored in directory: /tmp/pip-ephem-wheel-cache-t6rzhvda/wheels/89/ad/9e/afdd2aa93b530235733e958fe3b5e1af3fe0f6530eca3957fb
Successfully built dimentuna


In [2]:
import torch

In [ ]:
# torch.set_float32_matmul_precision('high')

In [3]:
from dimentuna import DTHfEncoder, LayerWrappebleDTHfLLM, LayerWrappebleQwen, DTConfig

In [4]:
from dimentuna import LinearMapper, DTLayerWrapper, LinearWrapper, FeedForwardProjector, LSTMProjector

In [5]:
from dimentuna import TwoPhasedTS, TwoPhasedSeq2SeqTS

In [ ]:
from dimentuna import DTMsgm

In [6]:
from datasets import load_dataset

In [7]:
from torch.utils.data import Dataset, DataLoader

# Parameters

In [8]:
batch_size = 4
epochs = 10

In [9]:
data_cache = '\data_cache'

In [10]:
model_cache = ".\model_cache"

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Datasets and Preprocessing

In [ ]:
mgsm = DTMsgm()

In [12]:
ds = load_dataset("nthakur/GSM8KInstruct-Parallel-instruct")

README.md:   0%|          | 0.00/838 [00:00<?, ?B/s]

(…)-00000-of-00001-d40eb5f467f27c86.parquet:   0%|          | 0.00/57.9M [00:00<?, ?B/s]

(…)-00000-of-00001-9ddeb1798398c38c.parquet:   0%|          | 0.00/3.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/69875 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3683 [00:00<?, ? examples/s]

In [13]:
train_ds = ds['train']
test_ds = ds['test']

In [14]:
train_df = train_ds.to_pandas()[["prompt", "code"]]
test_df = test_ds.to_pandas()[["prompt", "code"]]

In [15]:
train_df[["System_prompt", "Instruction"]] = train_df['prompt'].str.split('### Instruction:', expand=True)
test_df[["System_prompt", "Instruction"]] = test_df['prompt'].str.split('### Instruction:', expand=True)

In [16]:
train_df = train_df[["Instruction", "code"]]
test_df = test_df[["Instruction", "code"]]

In [17]:
train_df["Instruction"] = train_df["Instruction"].str.strip()
test_df["Instruction"] = test_df["Instruction"].str.strip()

In [18]:
train_df['Instruction'] = train_df['Instruction'].str.rsplit('?', n=1).str[0] + '?'
test_df["Instruction"] = test_df["Instruction"].str.rsplit('?', n=1).str[0] + '?'

In [ ]:
# Step 1: Add a column with the length of each "Instruction"
train_df["Instruction_Length"] = train_df["Instruction"].apply(len)
test_df["Instruction_Length"] = test_df["Instruction"].apply(len)

# Step 2: Determine a threshold (e.g., 95th percentile of length)
train_threshold = train_df["Instruction_Length"].quantile(0.95)
test_threshold = test_df["Instruction_Length"].quantile(0.95)

# Step 3: Filter out unusually large rows
train_df = train_df[train_df["Instruction_Length"] <= train_threshold]
test_df = test_df[test_df["Instruction_Length"] <= test_threshold]

# Step 4: Drop the temporary "Instruction_Length" column
train_df = train_df.drop(columns=["Instruction_Length"])
test_df = test_df.drop(columns=["Instruction_Length"])

In [19]:
train_df_english = train_df[train_df['code']=='en'].reset_index(drop=True)
test_df_english = test_df[test_df['code']=='en'].reset_index(drop=True)

In [20]:
train_df_thai = train_df[train_df['code']=='th'].reset_index(drop=True)
test_df_thai = test_df[test_df['code']=='th'].reset_index(drop=True)

In [21]:
train_df_english

,Instruction,code
0,John has 2 umbrellas in his house and 1 in the...,en
1,Brad wanted to set up a lemonade stand to earn...,en
2,"Angus, Patrick, and Ollie went fishing for tro...",en
3,Mary has a mystery book library. She started w...,en
4,Frank has an apple tree in his backyard. 5 app...,en
...,...,...
7094,If Jade earns $1600 per month and spent 75% of...,en
7095,John builds a toy bridge to support various we...,en
7096,"Coral reads 30 pages of a book on night 1, and...",en
7097,Janet has 24 dresses. Half of them have pocke...,en


In [22]:
test_df_english

,Instruction,code
0,Julian has 400 legos and wants to make lego mo...,en
1,"Mark has 30 candies, Peter has 25 candies, and...",en
2,"Marj has two $20 bills, three $5 bills, and $4...",en
3,Hunter saw 5 frogs sitting on top lily pads in...,en
4,A portable computer drive has enough kilobytes...,en
...,...,...
369,"Cappuccinos cost $2, iced teas cost $3, cafe l...",en
370,Jamal works at a library shelving books. He ha...,en
371,"Out of the 200 students in a class, thirty per...",en
372,Albert has to run 99 meters in gym class. The ...,en


In [23]:
test_df_english['Instruction'][0]

'Julian has 400 legos and wants to make lego models of two identical airplanes. If each airplane model requires 240 legos, how many more legos does Julian need?'

In [24]:
train_df_thai

,Instruction,code
0,ควินซีเพิ่งซื้อรถยนต์โดยใช้สินเชื่อ 5 ปีโดยไม่...,th
1,พีเตอร์กำลังเก็บใบไม้ร่วง ใช้เวลา 15 นาทีในการ...,th
2,บิลได้รับทีวีจอแบนที่มีขนาด 48 นิ้ว x 100 นิ้ว...,th
3,พอลได้รับเงิน 12.50 ดอลลาร์ต่อชั่วโมงที่เขาทำง...,th
4,เจมส์ซื้อเนื้อสเต็กแบบซื้อ 1 แถม 1 ราคาคือ 15 ...,th
...,...,...
7094,Jeanette กำลังฝึกการโยนลูกบอล ในแต่ละสัปดาห์เธ...,th
7095,ลอร่าต้องซื้อวัสดุประดับหน้าต่างสำหรับหน้าต่าง...,th
7096,โทมัสกำลังหวังที่จะวิ่งมาราธอนในปีหน้า ซึ่งมีร...,th
7097,เพจซื้อสติกเกอร์ใหม่และต้องการแบ่งปันกับเพื่อน...,th


In [25]:
test_df_thai['Instruction'][0]

'โครงการคอนโดใหม่นี้มีชั้นทั้งหมด 23 ชั้น ชั้นปกติมีหน่วยที่อยู่ 12 หน่วย ในขณะที่ชั้นพานเพนท์เฮาส์มีหน่วยเพียง 2 หน่วย เมื่อมีการจัดสรรชั้นบนสุด 2 ชั้นสำหรับหน่วยพานท์เฮาส์ จำนวนหน่วยทั้งหมดในคอนโดนี้เป็นเท่าใด?'

**DataLoaders**

In [26]:
train_english_loader = DataLoader(train_df_english['Instruction'], batch_size=batch_size)
test_english_loader = DataLoader(test_df_english["Instruction"], batch_size=batch_size)

In [27]:
train_thai_loader = DataLoader(train_df_thai['Instruction'], batch_size=batch_size//2)
test_thai_loader = DataLoader(test_df_thai['Instruction'], batch_size=batch_size//2)

In [28]:
xlmr_config = DTConfig(
    model_name="FacebookAI/xlm-roberta-base",
    cache_dir="./cache",          # Path to cache directory
    max_tokens=512,               # Maximum token limit
    temperature=0.7,              # Sampling temperature
    max_generation_length=256,    # Maximum length for generated text
    do_sample=True,               # Enable sampling
    truncation=True,              # Enable truncation
    padding=True,                 # Enable padding
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),  # Device configuration
    use_best_config=False,        # Use best configuration (custom logic)
    padding_side='right',         # Padding side
    output_hidden_states=False,   # Output hidden states
    output_attentions=False,      # Output attention values
    return_dict=True,             # Return output as dictionary
)

In [29]:
xlmr = DTHfEncoder(xlmr_config)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [30]:
test = ["This is test Sentence.", "This is test sentence 2.", "This is the longest test sentence 3."]

In [31]:
input_ten = xlmr.encode(test)

# Models

In [32]:
mapper = LinearMapper(1536, 1536, 1024)

In [34]:
qwen_model = LayerWrappebleQwen()

tokenizer_config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
qwen_model.set_system_prompt(mgsm.system_prompt)

In [ ]:
qwen_model.print_best_config()

In [35]:
qwen_ed = qwen_model.get_embeddings_size()

In [36]:
xlmr_ed = xlmr.get_embeddings_size()

In [37]:
projector = LSTMProjector(qwen_ed, xlmr_ed, 1024)

In [38]:
qwen_model.get_Layer_output(test, 1).shape

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


torch.Size([3, 9, 1536])

In [39]:
layer_1_wrapper= LinearWrapper(mapper)

In [40]:
qwen_model.replace_layer(1, layer_1_wrapper)

In [41]:
qwen_model.print_status()

-LLM
--Layer_0
---Layer_Frozen: False
--LayerWrapper_1
---Layer_Frozen: False
---Mapper_Frozen: False
---Engaged: True
---Alpha_Frozen: False Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: False
--Layer_3
---Layer_Frozen: False
--Layer_4
---Layer_Frozen: False
--Layer_5
---Layer_Frozen: False
--Layer_6
---Layer_Frozen: False
--Layer_7
---Layer_Frozen: False
--Layer_8
---Layer_Frozen: False
--Layer_9
---Layer_Frozen: False
--Layer_10
---Layer_Frozen: False
--Layer_11
---Layer_Frozen: False
--Layer_12
---Layer_Frozen: False
--Layer_13
---Layer_Frozen: False
--Layer_14
---Layer_Frozen: False
--Layer_15
---Layer_Frozen: False
--Layer_16
---Layer_Frozen: False
--Layer_17
---Layer_Frozen: False
--Layer_18
---Layer_Frozen: False
--Layer_19
---Layer_Frozen: False
--Layer_20
---Layer_Frozen: False
--Layer_21
---Layer_Frozen: False
--Layer_22
---Layer_Frozen: False
--Layer_23
---Layer_Frozen: False
--Layer_24
---Layer_Frozen: False
--Layer_25
---Layer_Frozen: False
--Layer_26
---Layer_Frozen: Fa

In [42]:
two_phased_ts = TwoPhasedSeq2SeqTS(
    llm=qwen_model,
    encoder=xlmr,
    projector=projector,
    train_loader=train_english_loader,
    val_loader=test_english_loader,
    lr=0.001,
    device=device,
    target_layers=[1],
    mapper_train_loader=train_thai_loader,
    mapper_val_loader=test_thai_loader
    layer_shift=3
)

In [43]:
qwen_model.freeze()

In [44]:
qwen_model.print_status()

-LLM
--Layer_0
---Layer_Frozen: True
--LayerWrapper_1
---Layer_Frozen: True
---Mapper_Frozen: True
---Engaged: True
---Alpha_Frozen: True Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: True
--Layer_3
---Layer_Frozen: True
--Layer_4
---Layer_Frozen: True
--Layer_5
---Layer_Frozen: True
--Layer_6
---Layer_Frozen: True
--Layer_7
---Layer_Frozen: True
--Layer_8
---Layer_Frozen: True
--Layer_9
---Layer_Frozen: True
--Layer_10
---Layer_Frozen: True
--Layer_11
---Layer_Frozen: True
--Layer_12
---Layer_Frozen: True
--Layer_13
---Layer_Frozen: True
--Layer_14
---Layer_Frozen: True
--Layer_15
---Layer_Frozen: True
--Layer_16
---Layer_Frozen: True
--Layer_17
---Layer_Frozen: True
--Layer_18
---Layer_Frozen: True
--Layer_19
---Layer_Frozen: True
--Layer_20
---Layer_Frozen: True
--Layer_21
---Layer_Frozen: True
--Layer_22
---Layer_Frozen: True
--Layer_23
---Layer_Frozen: True
--Layer_24
---Layer_Frozen: True
--Layer_25
---Layer_Frozen: True
--Layer_26
---Layer_Frozen: True
--Layer_27
---Layer_Froze

In [ ]:
q

In [ ]:
two_phased_ts.train(epochs)

Encoder_Frozen: True
Projector_Frozen: False
-LLM
--Layer_0
---Layer_Frozen: True
--LayerWrapper_1
---Layer_Frozen: True
---Mapper_Frozen: True
---Engaged: False
---Alpha_Frozen: True Value: 0.5 Not Used
--Layer_2
---Layer_Frozen: True
--Layer_3
---Layer_Frozen: True
--Layer_4
---Layer_Frozen: True
--Layer_5
---Layer_Frozen: True
--Layer_6
---Layer_Frozen: True
--Layer_7
---Layer_Frozen: True
--Layer_8
---Layer_Frozen: True
--Layer_9
---Layer_Frozen: True
--Layer_10
---Layer_Frozen: True
--Layer_11
---Layer_Frozen: True
--Layer_12
---Layer_Frozen: True
--Layer_13
---Layer_Frozen: True
--Layer_14
---Layer_Frozen: True
--Layer_15
---Layer_Frozen: True
--Layer_16
---Layer_Frozen: True
--Layer_17
---Layer_Frozen: True
--Layer_18
---Layer_Frozen: True
--Layer_19
---Layer_Frozen: True
--Layer_20
---Layer_Frozen: True
--Layer_21
---Layer_Frozen: True
--Layer_22
---Layer_Frozen: True
--Layer_23
---Layer_Frozen: True
--Layer_24
---Layer_Frozen: True
--Layer_25
---Layer_Frozen: True
--Layer_26
-

100%|██████████| 94/94 [00:18<00:00,  5.04it/s]


Epoch 0 : Loss 0.0027878659608787004 Eval Loss 0.0017115237919534457


100%|██████████| 94/94 [00:18<00:00,  5.05it/s]


Epoch 1 : Loss 0.0017144936932460733 Eval Loss 0.0017205070124622038


100%|██████████| 94/94 [00:18<00:00,  5.04it/s]


Epoch 2 : Loss 0.001618427458206113 Eval Loss 0.0015790199655345938


100%|██████████| 94/94 [00:18<00:00,  5.04it/s]


Epoch 3 : Loss 0.001575715510817376 Eval Loss 0.0014908197707437136


Epoch 4:  71%|███████   | 1253/1775 [04:30<02:17,  3.79it/s]

In [ ]:
qwen_model.print_status()

In [ ]:
qwen_model.engage_all_layer_wrappers(False)

In [ ]:
mgsm.evaluate(qwen_model,['en', 'th'], batch_size=batch_size, logging=True)

In [ ]:
qwen_model.engage_layer_wrapper(1)

In [ ]:
qwen_model.print_status()

In [ ]:
mgsm.evaluate(qwen_model,['en', 'th'], batch_size=batch_size, logging=True)